<h1 style="text-align:center;color:red;">The Lord of The Ring - Grammar</h1>
<p style="text-align:center;">By Maycon Cypriano Batestin</p>



### About the Dataset

The objective is to create a grammar checker in Portuguese based on the book Lord of the Rings by J.R.R. Tolkien

- **Source:** The book Lord of the Rings by J.R.R. Tolkien
- **Release:** Maycon Batestin
- **Licence:** Creative Commons Attribution-ShareAlike 4.0 International ([CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))



<h1 style="text-align:center;color:red;">Glossary</h1>


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
word                            						  |string     | a singular word of the book                             |
count                                                     |int        | the count time of word                               |
freq													  |float      | a frequence of the word           |






<h1 style="text-align:center;color:red;">Getting the Dataset </h1>


In [49]:
!clear
!python3 /Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/GRAMMAR/scripts/create_dataset.py

Traceback (most recent call last):
  File "/Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/GRAMMAR/scripts/create_dataset.py", line 22, in <module>
    text = open_book(url)
  File "/Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/GRAMMAR/scripts/create_dataset.py", line 14, in open_book
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)
  File "/Users/mayconcyprianobatestin/opt/anaconda3/lib/python3.9/site-packages/PyPDF2/_reader.py", line 1944, in __init__
    super().__init__(*args, **kwargs)
  File "/Users/mayconcyprianobatestin/opt/anaconda3/lib/python3.9/site-packages/PyPDF2/_reader.py", line 317, in __init__
    self.read(stream)
  File "/Users/mayconcyprianobatestin/opt/anaconda3/lib/python3.9/site-packages/PyPDF2/_reader.py", line 1374, in read
    self._basic_validation(stream)
  File "/Users/mayconcyprianobatestin/opt/anaconda3/lib/python3.9/site-packages/PyPDF2/_reader.py", line 1420, in _basic_validation
    raise PdfReadError(
PyPDF2.errors

<h1 style="text-align:center;color:red;">Librarys </h1>


In [109]:
import pandas as pd
import re
import nltk
import string
import PyPDF2
import requests
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords


nltk.download("punkt")



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mayconcyprianobatestin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [166]:
def df(path):
    df = pd.read_csv(path)
    return df

df = df("/Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/GRAMMAR/dataset/lotr.csv")
df

,word,count,freq
0,special,19,0.000032
1,note,26,0.000044
2,in,8775,0.014918
3,this,1664,0.002829
4,reprint,1,0.000002
...,...,...,...
16395,archival,1,0.000002
16396,utmost,1,0.000002
16397,xxxstmxxx,1,0.000002
16398,06,1,0.000002


In [167]:
def checkNAN(df):
    if df.isnull().values.any():
        df.dropna(inplace=True) 
        df.reset_index(drop=True, inplace=True)
        print("Checking for NaN values and fixing!.")
    else:
        print("There no NaN values on your dataset")

checkNAN(df)


Checking for NaN values and fixing!.


In [168]:
# Variables

text = " ".join([l for l in  df.word ])
token = set([word for word in nltk.tokenize.word_tokenize(text) if word.isalpha()])
alpha = string.ascii_letters
numbers = string.digits
puncts = string.punctuation
printable = string.printable
space = string.whitespace
token_string = "".join(token)


In [169]:
#create a test dataframe

def token_wrongs(token):
    if len(token) > 1:
        index = random.randint(0, len(token) - 1) 
        token_list = list(token)
        token_list.pop(index)
        return ''.join(token_list)
    else:
        return token


tuples = [(token, token_wrongs(token)) for token in token]

tId = list(enumerate(tuples))

test_df = pd.DataFrame(tId, tuples, columns=["id", 'test'])


test_df = list(test_df.test)
test_df

[('motor', 'otor'),
 ('furnace', 'furnac'),
 ('greed', 'gred'),
 ('beaklike', 'beklike'),
 ('hardly', 'hadly'),
 ('himsel', 'imsel'),
 ('htly', 'tly'),
 ('pps', 'pp'),
 ('eh', 'e'),
 ('suspense', 'suspnse'),
 ('suspected', 'suspeced'),
 ('bridled', 'bidled'),
 ('ges', 'ge'),
 ('goon', 'oon'),
 ('idleness', 'dleness'),
 ('stupid', 'stuid'),
 ('bodyguards', 'bodyguars'),
 ('unearthly', 'unearthl'),
 ('fallohides', 'allohides'),
 ('highland', 'higland'),
 ('reciting', 'reiting'),
 ('uruks', 'uruk'),
 ('advantage', 'advantge'),
 ('celebdil', 'ceebdil'),
 ('sawn', 'san'),
 ('object', 'objct'),
 ('tuesday', 'tueday'),
 ('forehead', 'frehead'),
 ('brightened', 'brghtened'),
 ('burnished', 'urnished'),
 ('heeding', 'heedng'),
 ('disappeared', 'disapeared'),
 ('goldwine', 'godwine'),
 ('stoo', 'sto'),
 ('tentacle', 'tenacle'),
 ('wafer', 'wafr'),
 ('walked', 'walkd'),
 ('matches', 'matche'),
 ('ping', 'pin'),
 ('leaflock', 'leaflck'),
 ('quieted', 'quiete'),
 ('tion', 'tin'),
 ('retentive', 're

In [170]:
def grammar(word):
    frequency = nltk.FreqDist(token)
    name = word.lower()
    slice = [ (name[:i], name[i:]) for i in range(len(name) + 1) ]
    new_word = [f"{r}{word}{l}" for r, l in slice for word in alpha.lower()]
    frequency = nltk.FreqDist(token)
    relative_frequency = {word: count / len(token) for word, count in frequency.items()}
    right = max(new_word, key=lambda word: relative_frequency.get(word, 0))
    return right
    


In [161]:
grammar("aragor")
#grammar_pop("aragornn")

'aragorn'

In [171]:
def acuracy(test_df):
    number_words = len(test_df)
    right = 0
    for r, w in test_df:
        right_word = grammar(w)
        if right_word == r:
            right = right + 1

    final = round(right * 100 / number_words, 3)

    return f"accuracy: {final}%"

acuracy(test_df)

'accuracy: 78.077%'